In [1]:
import os
import pandas as pd

# Define the paths to the directories containing the CSV files
dir_2_steps_rag = "results 2-steps RAG"
dir_common_rag = "results Common RAG"

# Function to read CSV files from a directory and add a 'rag_type' column
def read_and_label_csvs(directory, rag_type):
    dfs = []
    for filename in os.listdir(directory):
        if filename.endswith(".csv"):
            filepath = os.path.join(directory, filename)
            try:
                df = pd.read_csv(filepath, encoding='utf-8')
                df['rag_type'] = rag_type
                columns_to_remove = ['score', 'prompt_tokens', 'augmented_prompt_tokens', 'results_tokens', 'prompt_cost', 'results_cost', 'total_cost']
                df = df.drop(columns=[col for col in columns_to_remove if col in df.columns])
                dfs.append(df)
            except Exception as e:
                print(f"Error reading {filename}: {e}")
    return dfs

# Read and label CSV files from both directories
dfs_2_steps = read_and_label_csvs(dir_2_steps_rag, "2-steps RAG")
dfs_common = read_and_label_csvs(dir_common_rag, "Common RAG")

# Concatenate all DataFrames into a single DataFrame
all_dfs = dfs_2_steps + dfs_common

if all_dfs:
    final_df = pd.concat(all_dfs, ignore_index=True)
    
    # Reorder columns
    desired_order = ['model', 'attempt', 'temperature', 'top_p', 'rag_type', 'prompt', 'augmented_prompt', 'baseline', 'results']
    existing_columns = [col for col in desired_order if col in final_df.columns]
    final_df = final_df[existing_columns]

    print(final_df.head())
    print(final_df.rag_type.value_counts())
else:
    print("No CSV files found in the specified directories.")

                    model  attempt  temperature  top_p     rag_type  \
0  gpt-4o-mini-2024-07-18        1          0.1    0.1  2-steps RAG   
1  gpt-4o-mini-2024-07-18        1          0.1    0.5  2-steps RAG   
2  gpt-4o-mini-2024-07-18        1          0.1    0.9  2-steps RAG   
3  gpt-4o-mini-2024-07-18        1          1.0    0.1  2-steps RAG   
4  gpt-4o-mini-2024-07-18        1          1.0    0.5  2-steps RAG   

                                              prompt  \
0  Qual a descrição oficial do código NCM para a ...   
1  Qual a descrição oficial do código NCM para a ...   
2  Qual a descrição oficial do código NCM para a ...   
3  Qual a descrição oficial do código NCM para a ...   
4  Qual a descrição oficial do código NCM para a ...   

                                    augmented_prompt  \
0  \n    Você é um assistente especializado em re...   
1  \n    Você é um assistente especializado em re...   
2  \n    Você é um assistente especializado em re...   
3  \n    Voc

In [2]:
df = final_df[['model', 'attempt', 'temperature', 'top_p', 'rag_type', 'baseline', 'results']]
print(df)

                            model  attempt  temperature  top_p     rag_type  \
0          gpt-4o-mini-2024-07-18        1          0.1    0.1  2-steps RAG   
1          gpt-4o-mini-2024-07-18        1          0.1    0.5  2-steps RAG   
2          gpt-4o-mini-2024-07-18        1          0.1    0.9  2-steps RAG   
3          gpt-4o-mini-2024-07-18        1          1.0    0.1  2-steps RAG   
4          gpt-4o-mini-2024-07-18        1          1.0    0.5  2-steps RAG   
...                           ...      ...          ...    ...          ...   
18027  TeenyTinyLlama-160m-NCM-ft      196          1.0    0.5   Common RAG   
18028  TeenyTinyLlama-160m-NCM-ft      196          1.0    0.9   Common RAG   
18029  TeenyTinyLlama-160m-NCM-ft      196          1.9    0.1   Common RAG   
18030  TeenyTinyLlama-160m-NCM-ft      196          1.9    0.5   Common RAG   
18031  TeenyTinyLlama-160m-NCM-ft      196          1.9    0.9   Common RAG   

                                                bas

In [3]:
import os
from dotenv import load_dotenv
import openai
from tqdm import tqdm
import time

# Specify the path to your .env file
dotenv_path = "/mnt/4d4f90e5-f220-481e-8701-f0a546491c35/arquivos/projetos/.env"

# Load the .env file
load_dotenv(dotenv_path=dotenv_path)

# Access and store the environment variable
openai_api_key = os.getenv("OPENAI_API_KEY")
model = 'gpt-4o-mini-2024-07-18'

In [5]:
df.loc[:, 'comparison_prompt'] = final_df.apply(lambda row: f"""Avalie as respostas abaixo de 0 a 10, em que 0 representa discordância total e 10 concordância total, considerando os seguintes critérios:

        *Qualidade da Resposta*: A resposta é clara, concisa e bem estruturada ao responder quando comparada ao baseline?
        *Concordância*: A resposta faz sentido no contexto comparativo com o baseline?
        *Precisão/Acurácia*: A resposta apresenta informações corretas ao serem comparadas com o baseline?
        **Alucinação**: A resposta contém informações inventadas ou inconsistentes com a pergunta ou baseline, ou afirmações sem embasamento verificável? Penalize a nota se o conteúdo parecer falso ou fabricado.

        Resposta Base (Baseline): {row['baseline']}
        Resposta do Modelo: {row['results']}

        Atribua uma nota de 0 a 10 para a comparação entre as respostas (Baseline e Modelo), justificando brevemente sua avaliação. 
        Respostas do modelo que indicam falta de acesso a informações específicas e recomendam consultar fontes externas (ex: "Desculpe, mas não tenho acesso a informações específicas como CEPs...") devem ser penalizadas, pois não atingiram o objetivo.
        Retorne a avaliação no formato JSON, com as chaves "quality", "agreement", "accuracy", "hallucination" e "justification". 
        Não penalize a avaliação em caso de repetições no texto.
""", axis=1)

In [7]:
print(df['comparison_prompt'][0])
print(len(df))

Avalie as respostas abaixo de 0 a 10, em que 0 representa discordância total e 10 concordância total, considerando os seguintes critérios:

        *Qualidade da Resposta*: A resposta é clara, concisa e bem estruturada ao responder quando comparada ao baseline?
        *Concordância*: A resposta faz sentido no contexto comparativo com o baseline?
        *Precisão/Acurácia*: A resposta apresenta informações corretas ao serem comparadas com o baseline?
        **Alucinação**: A resposta contém informações inventadas ou inconsistentes com a pergunta ou baseline, ou afirmações sem embasamento verificável? Penalize a nota se o conteúdo parecer falso ou fabricado.

        Resposta Base (Baseline): A descrição correta de acordo com a Nomenclatura Comum do Mercusul para 'VASSOURAO CERPA PLAST FIO RIGIDO 370X63MM MAX' é Vassouras e escovas, mesmo constituindo partes de máquinas, de aparelhos ou de veículos, vassouras mecânicas de uso manual não motorizadas, pincéis e espanadores; cabeças prep

In [6]:
import json
from tqdm import tqdm

# Iterate over each row and make API call
output_filename = f"model scoring.csv"

# Initialize progress bar
for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing"):
    if index % 150 == 0 and index != 0:
        print("min. pause...")
        time.sleep(60)
    try:
        comparison_promt = row['comparison_prompt']
        
        response = openai.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": comparison_promt}],
            temperature=0.0
        )
        generated_text = response.choices[0].message.content
        
        # Process the JSON response
        metadata_str = generated_text.strip()
        if metadata_str.startswith("```json") and metadata_str.endswith("```"):
            metadata_str = metadata_str.strip("```json").strip()
        
        try:
            data = json.loads(metadata_str)
            # Update DataFrame with the extracted values
            df.loc[index, 'quality'] = data.get('quality')
            df.loc[index, 'agreement'] = data.get('agreement')
            df.loc[index, 'accuracy'] = data.get('accuracy')
            df.loc[index, 'hallucination'] = data.get('hallucination')
            df.loc[index, 'justification'] = data.get('justification')
        
        except json.JSONDecodeError as e:
            # Print error message if JSON decoding fails
            print(f"Error decoding JSON: {e}")
            print(f"Problematic JSON string: {metadata_str}")
            df.loc[index, 'quality'] = None
            df.loc[index, 'agreement'] = None
            df.loc[index, 'accuracy'] = None
            df.loc[index, 'hallucination'] = None
            df.loc[index, 'justification'] = f"JSON Error: {e}"

    except Exception as e:
        # Print error message if there's an issue processing the line
        print(f"Error processing line {index}: {e}")
        df.loc[index, 'quality'] = None
        df.loc[index, 'agreement'] = None
        df.loc[index, 'accuracy'] = None
        df.loc[index, 'hallucination'] = None
        df.loc[index, 'justification'] = f"Error: {e}"

# Save the updated DataFrame
df.to_csv(output_filename, index=False)

Processing:   1%|          | 150/18032 [07:11<13:50:22,  2.79s/it]

min. pause...


Processing:   2%|▏         | 300/18032 [14:34<12:20:46,  2.51s/it] 

min. pause...


Processing:   2%|▏         | 450/18032 [22:45<11:36:59,  2.38s/it] 

...

Processing:  99%|█████████▉| 17867/18032 [18:52:58<21:33,  7.84s/it]

Error decoding JSON: Expecting value: line 1 column 1 (char 0)
Problematic JSON string: Para fornecer uma avaliação adequada, preciso da resposta do modelo para comparar com a resposta base. Por favor, forneça a resposta do modelo para que eu possa realizar a avaliação.


Processing: 100%|█████████▉| 18000/18032 [19:12:03<03:43,  6.99s/it]

min. pause...


Processing: 100%|██████████| 18032/18032 [19:16:44<00:00,  3.85s/it]
